# Neural Network with Tensorflow

My first class  using the lower api of tensorflow. It should be improved with drop out regularization.

In [1]:
import pandas as pd
import  numpy as np
import tensorflow as tf

In [2]:
class neural_network:
    def __init__(self, layers,  epochs=1200,  mini_batch=2000, learning_rate=0.01 ):
        self.layers = layers
        self.epochs = epochs
        self.mini_batch = mini_batch 
        self.learning_rate = learning_rate
    
    def create_weights(self, layer,tipe= "fit"):
        if  tipe=="fit":
            
            return np.random.randn(self.layers_total[layer], self.layers_total[layer-1])
                               
        elif tipe== "predict":
            return self.weigths[layer-1]
  

    def create_bias(self, layer, tipe= "fit"):
        if  tipe=="fit":
            
            return np.zeros((self.layers_total[layer], 1))
                               
        elif tipe== "predict":
            return self.bias[layer-1]
        
    def forward(self, tipe="fit"):
        """Create the graph"""
        layers = self.layers_total
  
        tf.reset_default_graph()

        self.X_placeholder = tf.placeholder(dtype=tf.float32, name="x_placeholder")
        self.y_placeholder = tf.placeholder(dtype=tf.int32, name="y_placeholder")
        
        self.y_zeros = tf.one_hot(self.y_placeholder, self.n_categories,  dtype=tf.int32 )
        
        #y_zeros = np.zeros(( n ,  n_categories))
        #for j in range(n):
         #   y_zeros[j, int(y[j])] = 1
        self.weigths_graph = []
        self.bias_graph  = []
        self.z_graph = []
        self.a_graph =  []
        
        self.a_graph.append(self.X_placeholder)
        
        for l  in  range(1,len(layers)):
            self.weigths_graph.append(tf.Variable(self.create_weights(l, tipe) ,
                                                  dtype=tf.float32, name = "weight_{}".format(l)))
            self.bias_graph.append(tf.Variable(self.create_bias(l, tipe), 
                                               dtype=tf.float32,  name = "bias_{}".format(l)))
        for  l in range(1,len(layers)):
            self.z_graph.append(tf.add(tf.matmul(self.weigths_graph[l-1],self.a_graph[l-1]),
                                                self.bias_graph[l-1]) )
            
            if l+1< len(layers):
                self.a_graph.append(tf.nn.leaky_relu(self.z_graph[l-1]))
            
            elif tipe=="predict":
                self.a_graph.append(tf.nn.softmax(self.z_graph[l-1], axis=0))
        
        if tipe== "fit":
            self.cost =  tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(self.y_zeros, tf.transpose(self.z_graph[-1])))
            self.opt = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost)
    
   
    def fit(self, X, y):
            
        self.number_minibatchs = int(np.ceil(float(X.shape[0]) / self.mini_batch))
        
        X = X.T
        self.n_categories = len(np.unique(y))
        m ,  n = X.shape
        self.layers_total = [m] + self.layers + [self.n_categories]
        self.forward(tipe="fit")
        init = tf.global_variables_initializer()

        self.a =  []
        indices = np.arange(X.shape[1])
        with tf.Session() as sess:
            sess.run(init)
            for e in  range(self.epochs):
                np.random.shuffle(indices)
                for mini in range(self.number_minibatchs):
                    lower_limit = mini * self.mini_batch
                    upper_limit = (mini+1) * self.mini_batch

                  
                    sess.run(self.opt, feed_dict={self.X_placeholder:X[:,indices[lower_limit:upper_limit]], 
                                             self.y_placeholder:y[indices[lower_limit:upper_limit]]})

                
                if e%100==0:
                    temp1, temp2, temp3 = sess.run([self.cost,self.y_zeros, self.z_graph[-1]  ], 
                                                   feed_dict={self.X_placeholder:X,  self.y_placeholder:y})
                    print "costo es {}".format(temp1)

                    self.a.append(temp3)
                    
            self.weigths = [w.eval() for  w in self.weigths_graph]
            self.bias = [b.eval() for  b in self.bias_graph] 
    
    def predict(self,X):
        X = X.T
        self.forward(tipe="predict")
        init = tf.global_variables_initializer()
        
        with tf.Session()as sess:
            sess.run(init)
            prediccion = sess.run(self.a_graph[-1], feed_dict={self.X_placeholder:X})
        return prediccion


## Testing the class

loading data

In [3]:

from sklearn.datasets import fetch_mldata
from sklearn.model_selection import  train_test_split

mnist = fetch_mldata("MNIST original")
X_train , X_test,y_train, y_test = train_test_split(mnist.data, mnist.target)

X_train.shape ,y_train.shape , X_test.shape, y_test.shape

/home/javemar/miniconda2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)
/home/javemar/miniconda2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)


((52500, 784), (52500,), (17500, 784), (17500,))

In [4]:
test_nn = neural_network([25 ,20,18,16,10],epochs=2400)
test_nn.fit(X_train, y_train)

prediccion = np.argmax(test_nn.predict(X_train),  axis=0)
"El train score es {}".format((prediccion==y_train).sum() /float(y_train.shape[0]))

Instructions for updating:
Colocations handled automatically by placer.
costo es 13255.8310547
costo es 22.8697433472
costo es 31.929933548
costo es 1.75168693066
costo es 0.620983064175
costo es 0.590853154659
costo es 0.70061892271
costo es 0.317730754614
costo es 0.38599383831
costo es 0.280421286821
costo es 0.281893312931
costo es 0.224350377917
costo es 0.701605379581
costo es 0.252493679523
costo es 0.188815623522
costo es 0.129340276122
costo es 0.0738714635372
costo es 0.133665367961
costo es 0.0824574232101
costo es 0.0532941408455
costo es 0.0532832406461
costo es 0.027783587575
costo es 0.042276404798
costo es 0.0726992040873


'El train score es 0.991504761905'

Test Score

In [5]:

prediccion = np.argmax(test_nn.predict(X_test),  axis=0)
"El test score es {}".format((prediccion==y_test).sum() /float(y_test.shape[0]))

'El test score es 0.936342857143'